# matrix_factorization_similar

In [46]:
import os
import sys
import gc
import subprocess
import pickle
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import numpy as np
import cudf
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import SparseAdam

In [47]:
SEED = 42
random.seed(SEED)
cores = multiprocessing.cpu_count()

In [48]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [49]:
with open(PREP_DIR + 'matrix_factorization_embeddings.pkl', 'rb') as f:
    emb = pickle.load(f)

In [50]:
emb

array([[-1.2586755 , -0.9445114 , -1.306982  , ...,  0.9559919 ,
         1.9328526 , -1.1924242 ],
       [ 1.3087419 , -1.4055054 , -1.0826674 , ..., -1.187105  ,
         1.1438131 ,  1.2275976 ],
       [ 1.0434946 ,  0.94260514,  0.5085466 , ..., -0.54709625,
         0.52784127,  1.0566362 ],
       ...,
       [ 0.86217946, -0.6084551 , -1.2874287 , ..., -1.2562873 ,
         1.2235086 ,  0.9372563 ],
       [-0.6895318 , -0.62510663,  0.5771889 , ...,  1.0193185 ,
        -0.76250184, -0.73454607],
       [-1.1356103 , -1.2509845 ,  0.67123514, ...,  1.2281612 ,
        -1.0030451 , -1.367304  ]], dtype=float32)

In [51]:
from cuml.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=21, metric='euclidean')
knn.fit(emb)
_, aid_nns = knn.kneighbors(emb)
aid_nns = aid_nns[:, 1:]

In [52]:
aid_nns

array([[ 774566, 1573113, 1786445, ..., 1146374, 1127243, 1594312],
       [1611426,  417819,  365954, ..., 1550588,  148754, 1411052],
       [1307931, 1477707,  117228, ..., 1603274, 1301673, 1024351],
       ...,
       [ 878860, 1625770,  765637, ..., 1324275,  856323,  349859],
       [ 716607,   74137,  975872, ..., 1681194,  186877, 1512317],
       [ 627778,  621386, 1160970, ..., 1126215, 1694778,  445068]])

In [53]:
with open(PREP_DIR + 'matrix_factorization_embeddings_similar_top20.pkl', 'wb') as f:
    pickle.dump(aid_nns, f)